动态在pathlib中插入src

In [1]:
import sys
import os
from wcmatch import pathlib

display(sys.executable)
src_p = pathlib.Path("..") / "src"
if src_p.exists() and src_p.is_dir():
    print(f'insert {src_p.resolve()} in PythonPath')
    sys.path.insert(0, os.fspath(src_p.resolve()))

'/Users/lhc/.pyenv/versions/data/bin/python3.8'

insert /Users/lhc/dev/repo/prac_py/src in PythonPath


# 1 context manager

# 2 Function

In [1]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [2]:
def reverse(word):
    return word[::-1]
reverse('hello')
sorted(fruits,key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## 2.1 map()
特别是一个序列做map,直接用生成器表达式替换.

In [8]:
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

display(list(map(factorial, range(6))))
[factorial(n) for n in range(6) if n%2]

[1, 1, 2, 6, 24, 120]

[1, 6, 120]

## 2.2 函数内置属性

In [19]:
for n in dir(factorial):
    print(n,end='/')
print('\n')
fac = factorial
print(fac.__hash__())

__annotations__/__call__/__class__/__closure__/__code__/__defaults__/__delattr__/__dict__/__dir__/__doc__/__eq__/__format__/__ge__/__get__/__getattribute__/__globals__/__gt__/__hash__/__init__/__init_subclass__/__kwdefaults__/__le__/__lt__/__module__/__name__/__ne__/__new__/__qualname__/__reduce__/__reduce_ex__/__repr__/__setattr__/__sizeof__/__str__/__subclasshook__/

283942422


## 2.3 函数的参数
flutpy C5.7的拆解
### 仅限关键字键值对的函数

In [13]:
def f(a,*,b=1):
    return a,b
f(1,b=9)

(1, 9)

In [14]:
fac = factorial
display(f.__code__) # 属性非方法
display(f.__defaults__) # 为什么是None,仅限关键字参数?
display(fac.__defaults__)
f.__kwdefaults__

<code object f at 0x10750d3a0, file "<ipython-input-13-bd6c10d9a2f7>", line 1>

None

None

{'b': 1}

In [11]:
from inspect import signature
sig = signature(f)
display(str(sig))
for name,p in sig.parameters.items():
     print(f'{name}, default value:{p.default}, kind:{p.kind}')

'(a, *, b=1)'

a, default value:<class 'inspect._empty'>, kind:1
b, default value:1, kind:3


In [22]:
r = f(a=0,b=8)
r[0]

0

### star assignment,Py 3.5+
PEP 448,Additional Unpacking Generalizations / PEP 3132 -- Extended iterable unpacking

In [41]:
a=1,2
display(type(a),a)
*a,=[[1,2,3],[4,5,6]]
display(type(a),a)


tuple

(1, 2)

list

[[1, 2, 3], [4, 5, 6]]

In [73]:
# Whilst *elements, = iterable causes elements to be a list, elements = *iterable, causes elements to be a tuple.
# *b, 类似于函数定义 fun(*arg,**kwargs)), * in left side means catch everything that isn't assigned to a name and assign it to the starred expression.
*b,=3,4,5
print(type(b),b) 
*b,=(6,7,8)
print(type(b),b)
*b,c,d=3,4,5
print(type(b),b)

iterable = [1,2,3,4]
print(*iterable)
b = *iterable, #可看做 b=1,2,3,4,It expands the contents of the iterable it is attached to.
print(type(b),b)
b=[*iterable]
print(type(b),b)

<class 'list'> [3, 4, 5]
<class 'list'> [6, 7, 8]
<class 'list'> [3]
1 2 3 4
<class 'tuple'> (1, 2, 3, 4)
<class 'list'> [1, 2, 3, 4]


### *可变参数

In [74]:
def fun(a,*num):
    print(type(num),num)
fun(1) # 变长参数会转换成turple,这和*iterable是一直的,都是turple
fun(1,2,3,4) 
fun([1,2,3,4])

iterable = [1,2,3,4]
fun(*iterable)
iterable=([1,2],[3,4],5)
fun(*iterable)
iterable = {1:1,2:2,3:3,4:4} #*只解包dict的key
fun(*iterable)

<class 'tuple'> ()
<class 'tuple'> (2, 3, 4)
<class 'tuple'> ()
<class 'tuple'> (2, 3, 4)
<class 'tuple'> ([3, 4], 5)
<class 'tuple'> (2, 3, 4)


### ***关键字参数

In [78]:
def fun(**kw):
    print(kw)
fun(name='lhc',age='30')
iterable = {'one':1,'two':2,'three':3,'four':4} 
fun(**iterable)
iterable = {1:1,2:2,3:3,4:4} #fun() keywords must be strings,即变量名必须是strings
fun(**iterable)

{'name': 'lhc', 'age': '30'}
{'one': 1, 'two': 2, 'three': 3, 'four': 4}


TypeError: fun() keywords must be strings

### 可变参数*之前的参数不能指定参数名

In [52]:
def fun(a,*num):
    print(a,num)
fun(1,2,3)
fun(1,num=2) # TypeError: fun() got an unexpected keyword argument 'num'

1 (2, 3)


TypeError: fun() got an unexpected keyword argument 'num'

In [ ]:
fun(a=1,2,3) #SyntaxError: positional argument follows keyword argument

### 可变参数*之后的参数必须指定参数名

In [57]:
def fun(a,*b,c=None):
    print(a,b,c)
fun(1,2,3,4) # 若c没有默认值c=None:TypeError: fun() missing 1 required keyword-only argument: 'c'
fun(1,2,3,c=1)

1 (2, 3, 4) None
1 (2, 3) 1


### 关键字参数只能作为最后一个参数
关键字前面的参数按照位置赋值还是名称赋值都可以.  
**顺序：位置参数（必选参数），默认参数，单星号参数或星号分隔符，关键字参数，双星号参数；**  
默认参数和关键字参数很像,不要搞混,默认参数在加载时即赋值.

In [79]:
def fun(a,*b,c=None,**d):
    print(a,b,c,d)
fun(1,2,3,4,m=5,n=6)
fun(1)
fun(1,2)
fun(1,2,q=0)

1 (2, 3, 4) None {'m': 5, 'n': 6}
1 () None {}
1 (2,) None {}
1 (2,) None {'q': 0}


### 解包作为参数传入函数

In [70]:
def fun(a,b):
    print(a,b)
iterable = (1,2)
fun(*iterable)
iterable = [1,2,3,4]
fun(*iterable) # TypeError: fun() takes 2 positional arguments but 4 were given

1 2


TypeError: fun() takes 2 positional arguments but 4 were given

In [69]:
person1 = {'name':'lhc','age':30}
print("{name}'s age is {age}".format(**person1))

lhc's age is 30


## 2.4支持函数式编程的包
高阶函数演示.

In [50]:
from functools import reduce
from operator import mul

def fact_re(n):
    """ reduce 结合 mul 计算阶乘 """
    return reduce(mul,range(1,n+1))

fact_re(5)

120

### itemgetter()

In [2]:
from operator import itemgetter
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]
cc_name = itemgetter(1,0)
for c in metro_data:
     print(cc_name(c))

itemgetter(3)(metro_data[3])

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


(40.808611, -74.020386)

### attrgetter() and namedtuple()

In [4]:
from operator import attrgetter
from collections import namedtuple

# 参数1是namedtuple的值
LatLong = namedtuple('LL','lat long')
Metro = namedtuple('Metro','name cc pop corrd')
# name,cc,pop,(lat,long) 对元组进行拆包,并赋予名称
# 之后给到namedtuple:Metro
metro_area = [Metro(name,cc,pop,LatLong(lat,long)) for name,cc,pop,(lat,long) in metro_data]

attrgetter('name','corrd.lat','corrd.long')(metro_area[2])

('Mexico City', 19.433333, -99.133333)

### functools.partial

In [7]:
from operator import mul
from functools import partial

triple = partial(mul,3)
display(triple(7))
display(list(map(triple,range(1,11))))

21

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]

### function 实现 策略模式

In [14]:
from fluentpy.c6.order import Customer
from fluentpy.c6.order import LineItem
from fluentpy.c6.order import Order
from fluentpy.c6.order import fidelity_promo

lhc = Customer('LHC48052', 999)
cart = [LineItem('banana',4,0.5),LineItem('apple',3,0.4)]
display(Order(lhc,cart,fidelity_promo))

cart = [LineItem('orange',5,0.7)]
display(Order(lhc,cart,fidelity_promo))

cart = 'abc'
for i in cart:
    print(type(i))
    print(i)
cart = list(list(cart))
for i in cart:
    print(type(i))
    print(i)

fidelity:999


<Order total:3.20 due: 3.20>

fidelity:999


<Order total:3.50 due: 3.50>

<class 'str'>
a
<class 'str'>
b
<class 'str'>
c
<class 'str'>
a
<class 'str'>
b
<class 'str'>
c


# 3 Iterator and Generator
From Fluent Python Chapter 14.

## Iterable 和 Iterator

In [4]:
from fluentpy.c14.sentence import Sentence1

s = Sentence1("Crazy world, extraordinary world.")
display(s)
i = iter(s)
for word in i:
    print(word)

for word in s:
    print(word)

# i 中已无可迭代的内容.
next(i)

Sentence 'Crazy world,...dinary world.'

Crazy
world
extraordinary
world
Crazy
world
extraordinary
world


StopIteration: 

In [20]:
from fluentpy.c14.sentence import Sentence2

s = Sentence2("Crazy world, extraordinary world.")
for word in s:
    print(word)

# iter()每次构造新的迭代器
display(next(iter(s)))
display(next(iter(s)))

# 新的迭代器,重新开始
# while 不会捕捉stopiteration异常
i = iter(s)
while(True):
    display(next(i))



Crazy
world
extraordinary
world


'Crazy'

'Crazy'

'Crazy'

'world'

'extraordinary'

'world'

StopIteration: 

## 迭代器中使用生成器函数 yield

In [7]:
from fluentpy.c14.sentence import Sentence3

s = Sentence3("Sentence3, Crazy world, extraordinary world.")
for word in s:
    print(word)

next(iter(s))

in Sentence 3.
Sentence3
in Sentence 3.
Crazy
in Sentence 3.
world
in Sentence 3.
extraordinary
in Sentence 3.
world
in Sentence 3.


'Sentence3'

In [6]:
from fluentpy.c14.sentence import Sentence4

s = Sentence4("Sentence 4, Crazy world, extraordinary world.")
for word in s:
    print(word)


Sentence
4
Crazy
world
extraordinary
world
end
with
sentence
4


'Sentence'

## 生成器函数:yield from(广度和深度搜索)

In [5]:
from pycookbook.c4.node44 import Node

root = Node(0)
c1 = Node(1)
c2 = Node(2)
c3 = Node(3)

root.add_child(c1)
root.add_child(c2)
root.add_child(c3)

c1.add_child(Node(11))
c1.add_child(Node(12))
c2.add_child(Node(21))
c3.add_child(Node(31))

for ch in root.depth_first():
    print(f'out print:{ch}')

print('======================')
for ch in root.depth_first_yield_from():
    print(f'out print:{ch}')

print('======================')
isRoot = True
for ch in root.width_first(isRoot):
    print(f'out print:{ch}')
    isRoot = False

before yield:Node(0)
out print:Node(0)
after yield:Node(0)
in for Node(1)
before yield:Node(1)
Node(1)
after yield:Node(1)
in for Node(11)
before yield:Node(11)
Node(11)
after yield:Node(11)
in for Node(12)
before yield:Node(12)
Node(12)
after yield:Node(12)
in for Node(2)
before yield:Node(2)
Node(2)
after yield:Node(2)
in for Node(21)
before yield:Node(21)
Node(21)
after yield:Node(21)
in for Node(3)
before yield:Node(3)
Node(3)
after yield:Node(3)
in for Node(31)
before yield:Node(31)
Node(31)
after yield:Node(31)
inner print:before yield:Node(0)
out print:Node(0)
inner print:after yield:Node(0)
inner print:before yield:Node(1)
out print:Node(1)
inner print:after yield:Node(1)
inner print:before yield:Node(11)
out print:Node(11)
inner print:after yield:Node(11)
inner print:before yield:Node(12)
out print:Node(12)
inner print:after yield:Node(12)
inner print:before yield:Node(2)
out print:Node(2)
inner print:after yield:Node(2)
inner print:before yield:Node(21)
out print:Node(21)
inn

# 4. Decroator

## 4.1 装饰器加载即运行
函数的默认参数也是这个在装载时运行.

In [2]:
import fluentpy.c7.regi as re



run register(<function f1 at 0x10d7f0e50>)
run register(<function f2 at 0x10d877940>)


In [4]:
# 导入后registry即有值
display(re.registry)
# 装饰器返回的还是原函数,所以f1的元数据保持不变.
display(re.f1.__name__)
re.f1()

[<function fluentpy.c7.regi.f1()>, <function fluentpy.c7.regi.f2()>]

'f1'

running f1()


In [6]:
import fluentpy.c7.average as av

# make_sum_average()运行后,生产出average
mav = av.make_sum_average()
display(mav(10))
display(mav(15))
mav.__code__.co_varnames

10.0

12.5

('new_value', 'total')

In [4]:
mav.__code__.co_freevars

('series',)

In [5]:
# __closure__中保存着闭包的元素,本例中是series,cell_contents是series的值.
mav.__closure__[0].cell_contents

[10, 15]

In [16]:
mav = av.make_average()
mav(10)
mav(12)

11.0

In [10]:
def x1():
    count += 1
    return count
x1()

UnboundLocalError: local variable 'count' referenced before assignment

## 简单装饰器 timeit

In [3]:
import fluentpy.c7.usetimeit as ut

print('*'*20,'Calling snooze(0.5)')
ut.snooze(0.5)
print('*'*20,'Calling factorial(6),6!=')
ut.factorial(6)
# 装饰器没有使用@wraps(),函数原来的信息丢失.
print(f'After decroator,factorial --> {ut.factorial.__name__}')

******************** Calling snooze(0.5)
00.50406557 snooze(0.5) --> None
******************** Calling factorial(6),6!=
00.00000097 factorial(1) --> 1
00.00007083 factorial(2) --> 2
00.00012478 factorial(3) --> 6
00.00017559 factorial(4) --> 24
00.00022486 factorial(5) --> 120
00.00027752 factorial(6) --> 720
After decroator,factorial --> clocked


In [11]:
import fluentpy.c7.usetimeit as ut

ut.fibonacci(5)

00.00001494 fibonacci(1) --> 1
00.00000086 fibonacci(0) --> 0
00.00000080 fibonacci(1) --> 1
00.00008431 fibonacci(2) --> 1
00.00043836 fibonacci(3) --> 2
00.00000041 fibonacci(0) --> 0
00.00000043 fibonacci(1) --> 1
00.00323095 fibonacci(2) --> 1
00.00000070 fibonacci(1) --> 1
00.00000075 fibonacci(0) --> 0
00.00000091 fibonacci(1) --> 1
00.00006189 fibonacci(2) --> 1
00.00013084 fibonacci(3) --> 2
00.00362917 fibonacci(4) --> 3
00.00411098 fibonacci(5) --> 5


5

In [9]:
import fluentpy.c7.usetimeit as ut

ut.fibonacci_cache(10)

00.00000214 fibonacci_cache(10) --> 55


55

## 带参数的装饰器1
fluentp C7-22.  
注意闭包的自由变量registry作用域.

In [2]:
import fluentpy.c7.regi_param as regi_p

# 加载即运行,但装饰器每次一个新的register,导致registry没有累加
# 如果需要不断累加,像fluentpy C7.22中一样,把registry定义在module级别
# 这个主要想看下如何获取闭包的变量,如pycookbook C7.12

registry:4504258592 it's len->0
running register(active=False) -> decorate(<function f1 at 0x10c79cc10>)
complete regi, registry:4504258592 it's len->0
r:4504257696 it's len->0
registry:4504258592 it's len->0
running register(active=False) -> decorate(<function f2 at 0x10c7ad160>)
complete regi, registry:4504258592 it's len->0
r:4504257696 it's len->0
registry:4504258592 it's len->0
running register(active=True) -> decorate(<function f4 at 0x10c7ad700>)
complete regi, registry:4504258592 it's len->1
r:4504257696 it's len->1
registry:4504258592 it's len->1
running register(active=True) -> decorate(<function f5 at 0x10c7adc10>)
complete regi, registry:4504258592 it's len->2
r:4504257696 it's len->2


In [3]:
f = regi_p.register
display(f)
# 手动注册,一直使用装饰器的引用
deco = f(active=True)
display(deco)
f_run = deco(regi_p.f1)
display(f_run)
# run f1()
f_run()
display(len(deco.get_registry()))
f_run = deco(regi_p.f3)
display(len(deco.get_registry()))
# run f3()
f_run()

<function fluentpy.c7.regi_param.register(active=False, r={<function f4 at 0x10c7ad700>, <function f5 at 0x10c7adc10>})>

registry:4504258592 it's len->2


<function fluentpy.c7.regi_param.register.<locals>.decorate(func)>

running register(active=True) -> decorate(<function f1 at 0x10c79cc10>)
complete regi, registry:4504258592 it's len->3
r:4504257696 it's len->3


<function fluentpy.c7.regi_param.f1()>

running f1()


3

running register(active=True) -> decorate(<function f3 at 0x10c7ad040>)
complete regi, registry:4504258592 it's len->4
r:4504257696 it's len->4


4

running f3()


In [4]:
g = regi_p.register
deco = g(active=True)
f_run= deco(regi_p.f3)
f_run()

registry:4504258592 it's len->4
running register(active=True) -> decorate(<function f3 at 0x10c7ad040>)
complete regi, registry:4504258592 it's len->4
r:4504257696 it's len->4
running f3()


## 带参数的装饰器2
pycookbook C9.2和C9.4

In [2]:
from pycookbook.c9.logged import *
import logging

logging.basicConfig(level=logging.DEBUG)
x = add
display(x)
display(x.__wrapped__)
display(x(2,3))
x(4,5)

wraps(func)->functools.partial(<function update_wrapper at 0x10af9e310>, wrapped=<function add at 0x10cd42820>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
wrapper-><function logged.<locals>.decorate.<locals>.wrapper at 0x10cd428b0>
wraps(func)(wrapper)-><function add at 0x10cd428b0>
wraps(func)->functools.partial(<function update_wrapper at 0x10af9e310>, wrapped=<function spam at 0x10cd42d30>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
wrapper-><function logged.<locals>.decorate.<locals>.wrapper at 0x10cd42dc0>
wraps(func)(wrapper)-><function spam at 0x10cd42dc0>


<function pycookbook.c9.logged.add(x, y)>

<function pycookbook.c9.logged.add(x, y)>

DEBUG:pycookbook.c9.logged:add


5

DEBUG:pycookbook.c9.logged:add


9

解构logged这个装饰器的运行机制

In [3]:
from pycookbook.c9.logged import *
import logging

logging.basicConfig(level=logging.DEBUG)

def addIt(x,y):
    return x+y

logged(logging.CRITICAL)(addIt)(2,3)

f = logged(logging.CRITICAL)
display(f)
f = f(addIt)
# 由于@wraps的存在,这里保留了addIt的元信息.
# 否则返回: <function pycookbook.c9.logged.logged.<locals>.decorate.<locals>.wrapper(*args, **kwargs)>
# 如果返回了被decorate的func,就可以用实际参数进行调用了.
display(f)
f(3,4)

CRITICAL:__main__:addIt


<function pycookbook.c9.logged.logged.<locals>.decorate(func)>

<function __main__.addIt(x, y)>

CRITICAL:__main__:addIt


7

## 带参数的装饰器3
pycookbook C9.6

In [2]:
from pycookbook.c9.loggeds import *
import logging

logging.basicConfig(level=logging.DEBUG)

@loggeds
def addIt(x,y):
    return x+y



Condition func not None, execute loggeds with func...
func is None. get attatch_wrapper...
func is not None, execute attach_wrapper with func
func is None. get attatch_wrapper...
func is not None, execute attach_wrapper with func


In [3]:
display(addIt(2,3))
# log在函数定义时先执行了,等待addParam(3,3)的时候再执行.
print('*'*10)

@loggeds(level=logging.CRITICAL,message='addParam test',name='param example')
def addParam(x,y):
    return x+y

# 以下访问闭包属性
f = addParam
display(f.logmsg)
display(f(3,3))
f.set_msg('new_msg')
display('静态用法:',f.logmsg)
display('用函数:',f.msg_it())
display('动态用法:',f.get_msg())
display(f(4,4))

DEBUG:__main__:addIt


5

**********
Condition func=None,return func loggeds...
Condition func not None,return func loggeds...
get attatch_wrapper...
get attatch_wrapper...


'addParam test'

CRITICAL:param example:addParam test


...

'new_msg'

'动态用法:'

'new_msg'

CRITICAL:param example:new_msg


8

In [6]:
from pycookbook.c9.loggeds import *
import logging

loggeds运行时机和装饰器机制有关.见装饰器2的结构部分  
相当于loggeds(addIt),运行loggeds一次,func is not None

In [8]:
@loggeds
def addIt(x,y):
    return x+y

Condition func not None,return func loggeds...
get attatch_wrapper...
get attatch_wrapper...


In [9]:
addIt(3,4)

DEBUG:__main__:addIt


7

相当于loggeds(level=...)(addParam)  
第一次运行func is None,返回partial(loggeds,...)即还是loggeds,再次运行时func is not None

In [7]:

@loggeds(level=logging.CRITICAL,message='addParam test',name='param example')
def addParam(x,y):
    return x+y

Condition func=None,return func loggeds...
Condition func not None,return func loggeds...
get attatch_wrapper...
get attatch_wrapper...


## 将装饰器定义为类pycb C9.9

In [3]:
from pycookbook.c9.profiled import Profiled

@Profiled
def add(x,y):
    return x+y

in __init__(),wraps(func),f:functools.partial(<function update_wrapper at 0x104cc6310>, wrapped=<function add at 0x1081b6c10>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
**********
self: <pycookbook.c9.profiled.Profiled object at 0x1082cd340>
**********
in __init__(),f(self),f:<pycookbook.c9.profiled.Profiled object at 0x1082cd340>


In [4]:
add(3,4)

in __call__()...
ncalls:... 1


7

In [3]:
from pycookbook.c9.profiled import Profiled

def addIt(x,y):
    return x+y

p = Profiled(addIt)
print(f'***p:{p},p.__wrapped__:{p.__wrapped__}')
# p有__call__()函数,可调用,addIt(3,4)等同于p(3,4)
p(3,4)

in __init__(),wraps(func)->functools.partial(<function update_wrapper at 0x10af9e310>, wrapped=<function addIt at 0x10cd42e50>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
**********
self: <pycookbook.c9.profiled.Profiled object at 0x10cb6fc40>
**********
in __init__(),f(self)-><pycookbook.c9.profiled.Profiled object at 0x10cb6fc40>
***p:<pycookbook.c9.profiled.Profiled object at 0x10cb6fc40>,p.__wrapped__:<function addIt at 0x10cd42e50>
in __call__()...
ncalls:... 1


7

In [ ]:
from pycookbook.c9.profiled import Profiled
from functools import wraps

def addIt(x,y):
    return x+y

## 使用装饰器动态添加函数的参数 C9.11

In [2]:
from pycookbook.c9.option_debug import *

@optional_debug
def add(x,y):
    return x+y

In [3]:
display(add(2,3))

Calling add


5

In [4]:
import inspect
display(inspect.signature(add))

<Signature (x, y, *, debug=True)>

# 5. Object Oriented

## 函数参数是默认参数时
HauntedBus隐藏的bug.

In [3]:
from fluentpy.c8.HauntedBus import HauntedBus

# 初始化有参数时无任何问题
bus1 = HauntedBus(['lhc'])
display(bus1.passengers)
bus1.pick('gxt')
bus1.drop('lhc')
display(bus1.passengers)


['lhc']

['gxt']

In [5]:
bus2 = HauntedBus()
bus2.pick('haunt')
display(bus2.passengers)

# 函数默认值在定义函数时计算,即在module加载时进行
bus3 = HauntedBus()
display(bus3.passengers)

['haunt', 'haunt']

['haunt', 'haunt']

## SchoolBus 
正确的做法.

In [4]:
from fluentpy.c8.SchoolBus import SchoolBus

bus1 = SchoolBus()
bus1.pick('lhc')
display(bus1.passengers)

bus2 = SchoolBus()
bus2.pick('gcq')
display(bus2.passengers)

['lhc']

['gcq']

## Vector2d_v0: fulentpy C9.1
实现默认方法,不是每个类都需要这么实现,如果要借助内置方法如abs(),repr(),str()即可实现其方法,__iter__()使对象内容可迭代,即可用在类似于tuple()和list()之中.

In [6]:
import operator as op
from fluentpy.c9.vector2d_v0 import Vector2d

v = Vector2d(3,4)
display(abs(v))
display(op.eq(v,(3,4)))
display("{}({!r},{!r})".format(*[1,2]))

5.0

True

IndexError: Replacement index 2 out of range for positional args tuple

In [9]:
display(list(v))
len(list(v))

[3, 4]

2

In [50]:
v = Vector2d(3,4)
# 调用vector2d 自定义的__bytes__()
b = bytes(v)
display(b)

display(chr(b[0]))
display(b[1:])
v1 = Vector2d.frombytes(b)
display(v1)
display(v == v1)

***** in __bytes__()


b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

'a'

'd'

b'\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

Vector2d(3.0,4.0)

True

## @property

In [4]:
from pycookbook.c8.person import *

p = Person('coolfish')
display(p.first_name)
p.first_name='coolfish007'
display(p.first_name)

'coolfish'

'coolfish007'

In [5]:
from pycookbook.c8.person import *

p = Person(42) #TypeError: Expected a string

TypeError: Expected a string

### chr/bytearray/Py3 编码

In [80]:
# 默认编码是unicode_escape
display(chr(97))
display(chr(0x61)) #0x61==97
display(ord('你'))
display(chr(20320))
display(chr(0x4f60))# 两个8位

'a'

'a'

20320

'你'

'你'

In [60]:
display('abc'.encode('ascii'))
display('abc'.encode('utf-8'))
display('你好'.encode('utf-8'))
display('你好'.encode('unicode_escape'))

b'abc'

b'abc'

b'\xe4\xbd\xa0\xe5\xa5\xbd'

b'\\u4f60\\u597d'

In [59]:
display(b'abc'.decode('ascii'))

'abc'

In [45]:
from array import array

# typecode 不同,bytes()也不同
# b:1个byte / f:4个 / d:8个 / u:2个/
a = array('l',[3,4])
display(a)
b = bytes(a)
display(b)
x=3
x.to_bytes(8,'little')


array('l', [3, 4])

b'\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00'

b'\x03\x00\x00\x00\x00\x00\x00\x00'

In [47]:
b = bytes('hello world',encoding='iso8859-1')
display(b)
# 可见字节
b = bytes('hello world',encoding='ascii')
display(b,len(b))
b = bytes('你好',encoding='gbk')
display(b,len(b))
b = bytes('你好',encoding='utf-8')
display(b)

b'hello world'

b'hello world'

11

b'\xc4\xe3\xba\xc3'

4

b'\xe4\xbd\xa0\xe5\xa5\xbd'

### 函数体也有属性

In [7]:
def addIt(x,y):
    try:
        addIt.counter +=1
    except AttributeError:
        addIt.counter = 0
    print(f'execute {addIt.counter}')
    return x+y
addIt.a = 0
f = addIt
display(f(1,2))
print(f.a,f.counter)
addIt.a+=2
f = addIt
display(f(3,4))
print(f.a,f.counter)
addIt.__dict__

execute 0


3

0 0
execute 1


7

2 1


{'a': 2, 'counter': 1}